# PC Algorithm based MonitorRank

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict, OrderedDict

import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', FutureWarning)

import sys
sys.path.append("../")

from notebooklib.save import run_tsdr, save_tsdr, load_tsdr, load_tsdr_by_chaos

In [3]:
DATASET_ID = "9n6mf"

In [4]:
from meltria.priorknowledge.priorknowledge import SockShopKnowledge

pk = SockShopKnowledge(
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewares": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)

In [5]:
import functools
import multiprocessing

from diagnoser.diag import prepare_init_graph, fix_edge_directions_in_causal_graph, find_connected_subgraphs
from eval.groundtruth import check_cause_metrics
import diagnoser.metric_node as mn

# https://github.com/joblib/joblib/pull/366#issuecomment-267603530
def with_timeout(timeout):
    def decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            pool = multiprocessing.pool.ThreadPool(1)
            async_result = pool.apply_async(decorated, args, kwargs)
            try:
                return async_result.get(timeout)
            except multiprocessing.TimeoutError:
                record = args[1]
                print(f"Timeouted {record.chaos_case_full()}")
                pool.terminate()
                return
        return inner
    return decorator

# @with_timeout(10*60)  # 10 min timeout

## Overall evaluation

In [6]:
datasets = load_tsdr_by_chaos(
    DATASET_ID, suffix="fluxrank_pearsonr_medoid_only_ctnrs", revert_normalized_time_series=True,
)

In [7]:
from diagnoser import diag
from notebooklib import rank

In [12]:
import joblib

@with_timeout(10*60)  # 10 min timeout
def diagnose_and_rank(reduced_df, record, **kwargs):
    G, ranks = diag.build_and_walk_causal_graph(
        reduced_df, record.pk,
        enable_prior_knowledge=False,
        pc_library="cdt",
        cg_algo="pc",
        pc_citest_alpha=0.05,
        pc_citest="gaussian",
        pc_variant="stable",
        disable_orientation=False,
        disable_ci_edge_cut=False,
        walk_method="monitorrank",
        root_metric_type="latency",
    )
    if len(ranks) == 0:
        print(f"Failed to diagnose {record.chaos_case_full()} with {len(ranks)} ranks")
        return None
    return G, rank.create_rank_as_dataframe(ranks, DATASET_ID, record)

ranked_datasets = []
for (_, _), somethings_records in datasets.items():
    records = []
    for record, data_df_by_metric_type in somethings_records:
        reduced_df = pd.concat([
            data_df_by_metric_type["services"][-1],
            data_df_by_metric_type["containers"][-1],
            data_df_by_metric_type["middlewares"][-1],
        ], axis=1)
        records.append((record, reduced_df))

    results = joblib.Parallel(n_jobs=-1, backend="multiprocessing")(
        joblib.delayed(diagnose_and_rank)(reduced_df, record)
        for record, reduced_df in records
    )
    assert results is not None
    ranked_datasets.extend(result for result in results if result is not None)
ranked_datasets = [_[1] for _ in ranked_datasets]

In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    ranks_df = rank.create_rank_as_dataframe_for_multiple_cases_from_frames(ranked_datasets)
    display(ranks_df)

metric_name  \
dataset_id target_app chaos_type          chaos_comp   chaos_idx                                                   
9n6mf      sock-shop  pod-network-latency user-db      4                      c-orders-db_cpu_user_seconds_total   
                                                       4                        c-carts-db_fs_writes_bytes_total   
                                                       4                            c-payment_memory_usage_bytes   
                                                       4                           c-shipping_memory_usage_bytes   
                                                       4                                    c-payment_memory_rss   
                                                       4                       c-payment_cpu_usage_seconds_total   
                                                       4                                   s-shipping_throughput   
                                                       4                     c-carts-db_cpu_system_seconds_total   
                                                       4                         c-payment_memory_failures_total   
                                                       4                         c-payment_cpu_cfs_periods_total   
                                                       3                                     s-catalogue_latency   
                                                       3                            c-payment_memory_usage_bytes   
                                                       3             c-front-end_cpu_cfs_throttled_periods_total   
                                                       3             c-front-end_cpu_cfs_throttled_seconds_total   
                                                       3                                    s-payment_throughput   
                                                       3                                   c-front-end_last_seen   
                                                       3                                  s-front-end_throughput   
                                                       3                     c-front-end_cpu_usage_seconds_total   
                                                       3                                      s-carts_throughput   
                                                       3                                     c-front-end_sockets   
                                                       2                                       s-user_throughput   
                                                       2                                  s-catalogue_throughput   
                                                       2                       c-carts-db_cpu_user_seconds_total   
                                                       2                                          s-user_latency   
                                                       2                           c-user_memory_max_usage_bytes   
                                                       2                                         s-carts_latency   
                                                       2                                      s-carts_throughput   
                                                       2                         c-orders_cpu_user_seconds_total   
                                                       2                                     s-orders_throughput   
                                                       2                              c-carts-db_fs_writes_total   
                                                       1                                  s-catalogue_throughput   
                                                       1                                   s-shipping_throughput   
                                                       1                                         s-carts_latency   
                                         

In [14]:
rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10)

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
n

,#cases (metric),AC@K (metric),AVG@K (metric),#cases (container),AC@K (container),AVG@K (container),#cases (service),AC@K (service),AVG@K (service)
1,43,0.488372,0.488372,110,0.145455,0.145455,110,0.118182,0.118182
2,43,0.720930,0.604651,110,0.318182,0.231818,110,0.327273,0.222727
3,43,0.837209,0.682171,110,0.463636,0.309091,110,0.554545,0.333333
4,43,0.953488,0.750000,110,0.527273,0.363636,110,0.800000,0.450000
5,43,0.976744,0.795349,110,0.554545,0.401818,110,0.836364,0.527273
6,43,1.000000,0.829457,110,0.572727,0.430303,110,0.836364,0.578788
7,43,1.000000,0.853821,110,0.572727,0.450649,110,0.836364,0.615584
8,43,1.000000,0.872093,110,0.572727,0.465909,110,0.836364,0.643182
9,43,1.000000,0.886305,110,0.572727,0.477778,110,0.836364,0.664646
10,43,1.000000,0.897674,110,0.572727,0.487273,110,0.836364,0.681818


In [15]:
rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=True)

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, orders-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, user-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
n

#cases (metric)  AC@K (metric)  \
9n6mf sock-shop pod-cpu-hog         1                21       0.666667   
                                    2                21       0.904762   
                                    3                21       0.952381   
                                    4                21       1.000000   
                                    5                21       1.000000   
                                    6                21       1.000000   
                                    7                21       1.000000   
                                    8                21       1.000000   
                                    9                21       1.000000   
                                    10               21       1.000000   
                pod-memory-hog      1                20       0.300000   
                                    2                20       0.500000   
                                    3                20       0.700000   
                                    4                20       0.900000   
                                    5                20       0.950000   
                                    6                20       1.000000   
                                    7                20       1.000000   
                                    8                20       1.000000   
                                    9                20       1.000000   
                                    10               20       1.000000   
                pod-network-latency 1                 2       0.500000   
                                    2                 2       1.000000   
                                    3                 2       1.000000   
                                    4                 2       1.000000   
                                    5                 2       1.000000   
                                    6                 2       1.000000   
                                    7                 2       1.000000   
                                    8                 2       1.000000   
                                    9                 2       1.000000   
                                    10                2       1.000000   

                                        AVG@K (metric)  #cases (container)  \
9n6mf sock-shop pod-cpu-hog         1         0.666667                  32   
                                    2         0.785714                  32   
                                    3         0.841270                  32   
                                    4         0.880952                  32   
                                    5         0.904762                  32   
                                    6         0.920635                  32   
                                    7         0.931973                  32   
                                    8         0.940476                  32   
                                    9         0.947090                  32   
                                    10        0.952381                  32   
                pod-memory-hog      1         0.300000                  42   
                                    2         0.400000                  42   
                                    3         0.500000                  42   
                                    4         0.600000                  42   
                                    5         0.670000                  42   
                                    6         0.725000                  42   
                                    7         0.764286                  42   
                                    8         0.793750                  42   
                                    9         0.816667                  42   
                                    10        0.835000                  42   
                pod-network-latency 1         0.500000                  36   
                             

In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=False, group_by_cause_comp=True))

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no 

#cases (metric)  AC@K (metric)  \
9n6mf sock-shop carts        1                 4       0.000000   
                             2                 4       1.000000   
                             3                 4       1.000000   
                             4                 4       1.000000   
                             5                 4       1.000000   
                             6                 4       1.000000   
                             7                 4       1.000000   
                             8                 4       1.000000   
                             9                 4       1.000000   
                             10                4       1.000000   
                carts-db     1                 7       0.714286   
                             2                 7       0.857143   
                             3                 7       1.000000   
                             4                 7       1.000000   
                             5                 7       1.000000   
                             6                 7       1.000000   
                             7                 7       1.000000   
                             8                 7       1.000000   
                             9                 7       1.000000   
                             10                7       1.000000   
                catalogue    1                 4       0.500000   
                             2                 4       0.750000   
                             3                 4       0.750000   
                             4                 4       0.750000   
                             5                 4       0.750000   
                             6                 4       1.000000   
                             7                 4       1.000000   
                             8                 4       1.000000   
                             9                 4       1.000000   
                             10                4       1.000000   
                catalogue-db 1                 2       0.500000   
                             2                 2       0.500000   
                             3                 2       0.500000   
                             4                 2       1.000000   
                             5                 2       1.000000   
                             6                 2       1.000000   
                             7                 2       1.000000   
                             8                 2       1.000000   
                             9                 2       1.000000   
                             10                2       1.000000   
                orders       1                 4       0.500000   
                             2                 4       0.750000   
                             3                 4       1.000000   
                             4                 4       1.000000   
                             5                 4       1.000000   
                             6                 4       1.000000   
                             7                 4       1.000000   
                             8                 4       1.000000   
                             9                 4       1.000000   
                             10                4       1.000000   
                orders-db    1                 4       0.250000   
                             2                 4       0.250000   
                             3                 4       0.500000   
                             4                 4       0.750000   
                             5                 4       1.000000   
                             6                 4       1.000000   
                             7                 4       1.000000   
                             8                 4       1.000000   
                             9                 4       1.000000   
             

In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rank.create_localization_score_as_dataframe(ranked_datasets, pk, k=10, group_by_cause_type=True, group_by_cause_comp=True))

no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-network-latency, carts-db
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-cpu-hog, catalogue
no cause metrics: 9n6mf, sock-shop, pod-memory-hog, catalogue
no 

#cases (metric)  \
9n6mf sock-shop carts        pod-cpu-hog         1                 1   
                                                 2                 1   
                                                 3                 1   
                                                 4                 1   
                                                 5                 1   
                                                 6                 1   
                                                 7                 1   
                                                 8                 1   
                                                 9                 1   
                                                 10                1   
                             pod-memory-hog      1                 3   
                                                 2                 3   
                                                 3                 3   
                                                 4                 3   
                                                 5                 3   
                                                 6                 3   
                                                 7                 3   
                                                 8                 3   
                                                 9                 3   
                                                 10                3   
                             pod-network-latency 1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                                                 6                 0   
                                                 7                 0   
                                                 8                 0   
                                                 9                 0   
                                                 10                0   
                carts-db     pod-cpu-hog         1                 4   
                                                 2                 4   
                                                 3                 4   
                                                 4                 4   
                                                 5                 4   
                                                 6                 4   
                                                 7                 4   
                                                 8                 4   
                                                 9                 4   
                                                 10                4   
                             pod-memory-hog      1                 3   
                                                 2                 3   
                                                 3                 3   
                                                 4                 3   
                                                 5                 3   
                                                 6                 3   
                                                 7                 3   
                                                 8                 3   
                                                 9                 3   
                                                 10                3   
                             pod-network-latency 1                 0   
                                                 2                 0   
                                                 3                 0   
                                                 4                 0   
                                                 5                 0   
                     